In [1]:
!pip install --quiet "langchain" "langchain-community" "beautifulsoup4" "requests" "jinja2" "weasyprint"
!pip install -U --quiet langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.6/850.6 kB 40.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
d

In [2]:
!pip install --quiet \
    "numpy==1.26.4" \
    "scipy==1.13.1" \
    "scikit-learn==1.5.0" \
    "torch==2.3.1" \
    "torchaudio==2.3.1" \
    "torchvision==0.18.1" \
    "transformers==4.41.2" \
    "accelerate==0.30.1" \
    "bitsandbytes==0.43.1" \
    "diffusers==0.27.2" \
    "huggingface_hub==0.23.2" \
    "peft==0.10.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 48.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 98.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.0 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 103.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 100.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 17

In [3]:
import os
import gc

import torch
# import torch_xla
# import torch_xla.core.xla_model as xm

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from diffusers import StableDiffusionXLPipeline

from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_huggingface import HuggingFacePipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document

from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML

2025-09-07 15:33:00.583786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757259180.944201      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757259181.048434      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
HTML_TEMPLATE_STRING = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{ title }}</title>
    <style>
        /* Your CSS styles go here */
        body {
            font-family: sans-serif;
            line-height: 1.6;
            color: #333;
        }
        .container {
            max-width: 800px;
            margin: 20px auto;
            padding: 30px;
            background-color: #ffffff;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>{{ title }}</h1>
        <img src="{{ image_file }}" alt="{{ title }} Header Image">
        <p>{{ summary | replace('\\n', '<br>') | safe }}</p>
    </div>
</body>
</html>
"""

In [5]:
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# device = xm.xla_device()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"We are now using= {device}")

We are now using= cuda


In [22]:
def scrape_from_web(url, file_path = "content.txt"):
    print("Step 1 initialized, Scraping content from the link")

    try:
        loader = WebBaseLoader(url)
        docs = loader.load()

        content = "\n".join([doc.page_content for doc in docs])

        with open(file_path, 'w', encoding="utf-8") as f:
            f.write(content)

        print(f"Content saved to {file_path}")
        return content

    except Exception as e:
        print(f"Error scraping URL= {e}")
        return None

In [27]:
def summarize(content_path = "content.txt", summary_path = "summary.txt", title_path = "title.txt", image_prompt_path = "image_prompt.txt"):
    print("\nStep 2 initialized, Summarizing content")

    model = None
    tokenizer = None
    pipe = None
    llm = None
    try:
        with open(content_path, 'r', encoding="utf-8") as f:
            content = f.read()

        model_id = "microsoft/Phi-3-mini-128k-instruct"

        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(model_id, 
                                                     trust_remote_code=True, 
                                                     device_map="auto", 
                                                     load_in_8bit=True
                                                    )
        
        pipe = pipeline("text-generation", 
                        model = model, 
                        tokenizer = tokenizer,
                        max_new_tokens = 1200,
                        eos_token_id = tokenizer.eos_token_id,
                        do_sample = True,
                        temperature = 0.8,
                        repetition_penalty = 1.15
                       )
        
        llm = HuggingFacePipeline(pipeline = pipe)

        if len(content) > 30000:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size = 15000, chunk_overlap = 500)
            split_docs = text_splitter.create_documents([content])
            print(f"Article is splitted into {len(split_docs)} chunks")
            
            map_prompt_template = "Summarize the key points of this text chunk:\n\n{text}\n\nCONCISE SUMMARY:"
            map_prompt = PromptTemplate.from_template(map_prompt_template)
            combine_prompt_template = "You are a newsletter editor. Combine these summaries into one cohesive, final summary. The final summary should be engaging and between 500 and 1000 words.\n\nSUMMARIES:\n{text}\n\nFINAL SUMMARY:"
            combine_prompt = PromptTemplate.from_template(combine_prompt_template)
            
            summary_chain = load_summarize_chain(llm = llm, 
                                                 chain_type = "map_reduce",
                                                 map_prompt = map_prompt,
                                                 combine_prompt = combine_prompt,
                                                 verbose = True
                                                )
            
            summary = summary_chain.invoke(split_docs)['output_text']

        else:
            docs = [Document(page_content = content)]
            
            prompt_template = "You are a newsletter editor. Write a cohesive, final summary of the following text. The final summary should be engaging and between 500 and 1000 words.\n\nTEXT:\n{text}\n\nFINAL SUMMARY:"
            prompt = PromptTemplate.from_template(prompt_template)
            
            summary_chain = load_summarize_chain(llm=llm,
                                                 chain_type="stuff",
                                                 prompt=prompt)
            summary = summary_chain.invoke(docs)['output_text']

        summary_clean = summary.split("FINAL SUMMARY:")[-1].strip()
        
        with open(summary_path, 'w', encoding="utf-8") as f:
            f.write(summary_clean)

        print(f"Full Summary saved to {summary_path}")

        print("Step 3 initialized. Generating title")

        title_messages = [
            {"role": "user", "content": f"Based on this summary, generate only one extremely catchy but concise newsletter title. The title must be under 12 words.\n\nSUMMARY:\n{summary}"}
        ]

        title_outputs = pipe(title_messages, max_new_tokens = 50)

        generated_title = title_outputs[0]['generated_text'][-1]['content']
        final_title = generated_title.strip().split('\n')[0].strip().lstrip('1234567890.-* ')
        
        with open(title_path, 'w', encoding="utf-8") as f:
            f.write(final_title)

        print(f"title saved to {title_path}")

        print("Step 4 initialized. Generating image prompt")

        prompt_messages = [{"role": "user", "content": f"Based on the following text, create a short, visually descriptive prompt for an AI image generator. Focus on concrete objects, scenes, colors, and styles. Do not include abstract concepts. Maximum 60 words.\n\nTEXT:\n{summary}"}]
        prompt_outputs = pipe(prompt_messages, max_new_tokens=100)
        generated_prompt = prompt_outputs[0]['generated_text'][-1]['content']
        final_image_prompt = generated_prompt.strip().replace('"', '')
        
        with open(image_prompt_path, "w", encoding="utf-8") as f:
            f.write(final_image_prompt) 

        print(f"image prompt saved to {image_prompt_path}")
        return summary, final_title, final_image_prompt

    finally:
        print("Cleaning up summarization model from memory")
        del model
        del tokenizer
        del pipe
        del llm
        clear_memory()

In [28]:
def generate_newsletter_image(image_prompt_path = "image_prompt.txt", title_path = "title.txt", image_path = "newsletter_image.png"):
    print("\nStep 5 initialized, Generating image for newsletter")

    pipe = None
    try:
        with open(image_prompt_path, 'r', encoding="utf-8") as f:
            visual_prompt = f.read().strip()
        with open(title_path, 'r', encoding="utf-8") as f:
            title = f.read()

        final_prompt = f"Digital art for a newsletter titled '{title}'. A clean, professional illustration of: {visual_prompt}. Marketing aesthetic, vibrant colors."
        
        pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype = torch.float16,
            variant = "fp16",
            use_safetensors = True
        )

        pipe = pipe.to(device)

        image = pipe(prompt = final_prompt).images[0]
        image.save(image_path)
        print(f"Image saved to {image_path}")

        return image_path

    finally:
        print("Cleaning up diffusion model from memory")
        del pipe
        clear_memory()

In [29]:
def assemble(summary_path, image_path, title_path, output_name="newsletter"):
    print("Step 4 initialized, Assembling final Newsletter")

    try:
        with open(summary_path, 'r', encoding="utf-8") as f:
            summary_text = f.read()
        with open(title_path, 'r', encoding="utf-8") as f:
            title = f.read().strip()

        template = Environment().from_string(HTML_TEMPLATE_STRING)

        html_content = template.render(
            title = title,
            summary = summary_text,
            image_file = image_path
        )

        html_file_path = f"{output_name}.html"

        with open(html_file_path, 'w', encoding="utf-8") as f:
            f.write(html_content)

        print(f"HTML Newsletter saved to {html_file_path}")

    except Exception as e:
        print(f"An error occured during assembly= {e}")

In [30]:
def run_full_pipeline(url):
    print("Starting automated Newsletter Generation")

    content_file = "content.txt"
    summary_file = "summary.txt"
    title_file = "title.txt"
    image_prompt_file = "image_prompt.txt"
    image_file = "newsletter_image.png"

    scrape_from_web(url, content_file)
    summarize(content_file, summary_file, title_file, image_prompt_file)
    generate_newsletter_image(image_prompt_file, title_file, image_file)
    assemble(summary_file, image_file, title_file)

    print("\n Pipeline ended, newsletter is created in kaggle/working/ folder")

article_url = "https://www.theverge.com/24172762/sonos-ace-headphones-video-review"
run_full_pipeline(article_url)

Starting automated Newsletter Generation
Step 1 initialized, Scraping content from the link


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Content saved to content.txt

Step 2 initialized, Summarizing content


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (102 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['people conversing amiably inside coffee houses accompanied by warm amber neon signs proclaiming wholehe. marketing aesthetic, vibrant colors.']
Token indices sequence length is longer than the specified maximum sequence length for this model (102 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['people conversing amiably inside coffee houses accompanied by warm amber neon signs proclaiming wholehe. marketing aesthetic, vibrant colors.']


  0%|          | 0/50 [00:00<?, ?it/s]

Image saved to newsletter_image.png
Cleaning up diffusion model from memory
Step 4 initialized, Assembling final Newsletter
HTML Newsletter saved to newsletter.html

 Pipeline ended, newsletter is created in kaggle/working/ folder
